In [ ]:
import os
import numpy as np
from tqdm import tqdm

In [ ]:
from avapi.carla import CarlaScenesManager


cpath = os.path.join("/data/shared/CARLA/multi-agent-intersection/")
CSM = CarlaScenesManager(cpath)
idx = 0
CDM = CSM.get_scene_dataset_by_index(idx)
vid_folder = f"videos_intersection_{idx}"

print(CSM.scenes)
print(f"{len(CDM)} frames")

In [ ]:
import matplotlib.pyplot as plt
from avapi.visualize.snapshot import show_lidar_bev_with_boxes

# load point cloud
lidar_sensor = "lidar-0"
agent = 0
frame_idx = 10
frame = CDM.get_frames(sensor=lidar_sensor, agent=agent)[frame_idx]
pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
show_lidar_bev_with_boxes(pc=pc)

# run concave hull algorithm
hull = pc.concave_hull_bev()
plt.plot(-1 * hull[:, 1], hull[:, 0])
plt.axis("scaled")
plt.show()

## Test Multi-Agent Tracking and Fusion

In [ ]:
from avstack.geometry import (
    Sphere,
    GlobalOrigin3D,
    transform_orientation,
    Box3D,
    Position,
    Attitude,
)
from avstack.calibration import LidarCalibration
from avstack.modules.perception.detections import BoxDetection
from avstack.modules.perception.object3d import MMDetObjectDetector3D
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D
from avstack.modules.tracking.stonesoup import StoneSoupKalmanTracker3DBox
from avstack.modules.tracking.multisensor import MeasurementBasedMultiTracker

from avstack.datastructs import DataContainer


# init models
# agents = list(range(len(CDM.get_agents(frame=1))))
agents = list(range(4))
agent_is_static = {
    i: "static" in CDM.get_agent(frame=1, agent=i).obj_type for i in agents
}
n_static = sum(list(agent_is_static.values()))
print(
    "There are {} agents\n   {} mobile, {} static".format(
        len(agents), len(agents) - n_static, n_static
    )
)
percep_veh = MMDetObjectDetector3D(model="pointpillars", dataset="carla-vehicle", gpu=0)
percep_inf = MMDetObjectDetector3D(
    model="pointpillars", dataset="carla-infrastructure", gpu=0
)
percep_col = None
# trackers = {agent: BasicBoxTracker3D() for agent in agents}
trackers = {agent: StoneSoupKalmanTracker3DBox() for agent in agents}
trackers["central"] = MeasurementBasedMultiTracker(tracker=BasicBoxTracker3D())
trackers["collab"] = BasicBoxTracker3D()

# init data structures
dets = {}
tracks = {}
imgs_all = {agent: [] for agent in agents}
pcs_all = {agent: [] for agent in agents}
dets_all = {agent: [] for agent in agents}
dets_all["collab"] = []
tracks_all = {agent: [] for agent in agents}
tracks_all["central"] = []
tracks_all["collab"] = []
agent_0_frames = CDM.get_frames(sensor="lidar-0", agent=0)[1:-1]
platforms_all = {agent: [] for agent in agents}

# flags for this run
run_distributed_perception = True
run_distributed_tracking = True
run_centralized_tracking = True
run_collaborative_perception = False
run_collaborative_tracking = False

# run loop
n_frames_max = 2000
ego_agent = agents[0]
for frame in tqdm(agent_0_frames[: min(n_frames_max, len(agent_0_frames))]):
    found_data = False
    fovs = {}
    platforms = {}
    perception_input = {}
    for agent in agents:
        ###############################################
        # GET DATA
        ###############################################
        lidar_sensor = "lidar-0"
        camera_sensor = "camera-0"
        img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
        pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
        imgs_all[agent].append(img)
        pcs_all[agent].append(pc)
        objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
        calib = CDM.get_calibration(frame=frame, sensor=lidar_sensor, agent=agent)
        fovs[agent] = pc.concave_hull_bev()
        # fovs[agent] = Sphere(radius=100)
        platforms[agent] = calib.reference
        platforms_all[agent].append(calib.reference)

        ###############################################
        # DISTRIBUTED PERCEPTION
        ###############################################
        found_data = True
        if run_distributed_perception:
            if agent_is_static[agent]:
                dets[agent] = percep_inf(pc)
            else:
                dets[agent] = percep_veh(pc)
            dets_all[agent].append(dets[agent])

        ###############################################
        # DISTRIBUTED TRACKING USING DISTRIBUTED PERCEP
        ###############################################
        if run_distributed_tracking:
            assert run_distributed_perception
            tracks[agent] = trackers[agent](
                dets[agent], platform=calib.reference, calibration=calib
            )
            if not isinstance(tracks[agent], DataContainer):
                raise
            tracks_all[agent].append([track.box3d.copy() for track in tracks[agent]])

    ###############################################
    # COLLABORATIVE PERCEPTION/TRACKING
    ###############################################
    if run_collaborative_perception:
        raise NotImplementedError

    ###############################################
    # CENTRALIZED TRACKING USING DISTRIBUTED PERCEP
    ###############################################
    # run central tracker on all detections
    if found_data:
        if run_centralized_tracking:
            # Run trust model

            # Run tracking
            tracks["central"] = trackers["central"](
                detections=dets,
                fovs=fovs,
                platforms=platforms,
            )
            tracks_all["central"].append(tracks["central"])

## Visualize

In [ ]:
from utils import make_agent_movies, make_central_movie, make_collab_movie

# agent-specific movie
if run_distributed_tracking:
    print("Making distributed agent movies")
    extent = [[-70, 70], [-70, 70], [-100, 100]]
    for agent in agents:
        make_agent_movies(
            imgs=imgs_all[agent],
            pcs=pcs_all[agent],
            dets=dets_all[agent],
            tracks=tracks_all[agent],
            agent=agent,
            extent=extent,
            percep_movies=False,
            track_movies=True,
            img_movies=True,
            bev_movies=True,
            vid_folder=vid_folder,
        )


# central tracking movie
if run_centralized_tracking:
    print("Making centralized tracking movies")
    extent = [[-150, 20], [-80, 40], [-100, 100]]
    make_central_movie(
        pcs_all,
        tracks_all["central"],
        ego=platforms_all[ego_agent],
        extent=extent,
        vid_folder=vid_folder,
        colormethod="channel-4",
    )


# collaborative perception movie
if run_collaborative_tracking:
    print("Making collaborative tracking movies")
    extent = [[-150, 20], [-80, 40], [-100, 100]]
    make_collab_movie(
        pcs_all,
        tracks_all["collab"],
        ego=platforms_all[ego_agent],
        extent=extent,
        vid_folder=vid_folder,
        colormethod="channel-4",
    )

In [ ]:
# from avstack.geometry import (
#     Sphere,
#     GlobalOrigin3D,
#     transform_orientation,
#     Box3D,
#     Position,
#     Attitude,
# )
# from avstack.calibration import LidarCalibration
# from avstack.modules.perception.detections import BoxDetection
# from avstack.modules.perception.object3d import MMDetObjectDetector3D
# from avstack.modules.tracking.tracker3d import BasicBoxTracker3D
# from avstack.modules.tracking.multisensor import MeasurementBasedMultiTracker

# from avstack.datastructs import DataContainer


# # init models
# # agents = list(range(len(CDM.get_agents(frame=1))))
# agents = list(range(4))
# agent_is_static = {
#     i: "static" in CDM.get_agent(frame=1, agent=i).obj_type for i in agents
# }
# percep_veh = MMDetObjectDetector3D(model="pointpillars", dataset="carla-vehicle", gpu=0)
# percep_inf = MMDetObjectDetector3D(model="pointpillars", dataset="carla-infrastructure", gpu=0)
# percep_col = None
# trackers = {agent: BasicBoxTracker3D() for agent in agents}
# trackers["central"] = MeasurementBasedMultiTracker(tracker=BasicBoxTracker3D())
# trackers["collab"] = BasicBoxTracker3D()

# # init data structures
# dets = {}
# tracks = {}
# imgs_all = {agent: [] for agent in agents}
# pcs_all = {agent: [] for agent in agents}
# dets_all = {agent: [] for agent in agents}
# dets_all["collab"] = []
# tracks_all = {agent: [] for agent in agents}
# tracks_all["central"] = []
# tracks_all["collab"] = []
# agent_0_frames = CDM.get_frames(sensor="lidar-0", agent=0)[1:-1]
# platforms_all = {agent: [] for agent in agents}

# # flags for this run
# run_distributed_perception = True
# run_distributed_tracking = True
# run_centralized_tracking = True
# run_collaborative_perception = False
# run_collaborative_tracking = False

# # run loop
# n_frames_max = 2000
# ego_agent = agents[0]
# for frame in tqdm(agent_0_frames[: min(n_frames_max, len(agent_0_frames))]):
#     found_data = False
#     fovs = {}
#     platforms = {}
#     perception_input = {}
#     for agent in agents:
#         ###############################################
#         # GET DATA
#         ###############################################
#         lidar_sensor = "lidar-0"
#         camera_sensor = "camera-0"
#         img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
#         pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
#         imgs_all[agent].append(img)
#         pcs_all[agent].append(pc)
#         objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
#         calib = CDM.get_calibration(frame=frame, sensor=lidar_sensor, agent=agent)
#         fovs[agent] = Sphere(radius=100)
#         platforms[agent] = calib.reference
#         platforms_all[agent].append(calib.reference)

#         ###############################################
#         # DISTRIBUTED PERCEPTION
#         ###############################################
#         found_data = True
#         if run_distributed_perception:
#             if agent_is_static[agent]:
#                 dets[agent] = percep_inf(pc)
#             else:
#                 dets[agent] = percep_veh(pc)
#             dets_all[agent].append(dets[agent])

#         ###############################################
#         # DISTRIBUTED TRACKING USING DISTRIBUTED PERCEP
#         ###############################################
#         if run_distributed_tracking:
#             assert run_distributed_perception
#             tracks[agent] = trackers[agent](dets[agent], platform=calib.reference)
#             if not isinstance(tracks[agent], DataContainer):
#                 raise
#             tracks_all[agent].append([track.box3d.copy() for track in tracks[agent]])

#         ###############################################
#         # PREP FOR COLLABORATIVE PERCEPTION
#         ###############################################
#         # Construct the input data structure for collaborative perception model
#         ref_new = pc.reference.get_ground_projected_reference()
#         ref_new.x[2] += 1.8  # position as a normal lidar sensor
#         calib_new = LidarCalibration(reference=ref_new)
#         pc_proj = pc.project(calib_new)
#         ref_global = pc_proj.calibration.reference.integrate(start_at=GlobalOrigin3D)
#         position = ref_global.x
#         attitude = ref_global.q
#         euler = transform_orientation(attitude, "quat", "euler")
#         lidar_pose = np.asarray(
#             [
#                 *position,
#                 np.degrees(euler[0]),
#                 np.degrees(euler[2]),
#                 np.degrees(euler[1]),
#             ]
#         )
#         perception_input[agent] = {
#             "lidar": pc_proj.data.x,
#             # This pose is in OPV2V format (x, y, z, roll, yaw, pitch) and the rotation are in degrees not radians.
#             "lidar_pose": lidar_pose,
#         }

#     ###############################################
#     # COLLABORATIVE PERCEPTION/TRACKING
#     ###############################################
#     if run_collaborative_perception:
#         pred_bboxes, _ = percep_col.run(perception_input, ego_agent)
#         collab_dets = []
#         for pred_bbox in pred_bboxes:
#             det = BoxDetection(
#                 "lidar-0",
#                 Box3D(
#                     position=Position(x=pred_bbox[0:3], reference=platforms[ego_agent]),
#                     # Note that Carla coordination has a reverted y axis (?). Taking a negation of yaw seems make it right.
#                     attitude=Attitude(
#                         q=transform_orientation([0, 0, -pred_bbox[6]], "euler", "quat"),
#                         reference=platforms[ego_agent],
#                     ),
#                     hwl=pred_bbox[3:6][::-1],
#                 ),
#                 platforms[ego_agent],
#             )
#             collab_dets.append(det)
#         collab_dets = DataContainer(
#             data=collab_dets, frame=frame, timestamp=0, source_identifier="collab"
#         )
#         dets_all["collab"].append(collab_dets)

#     if run_collaborative_tracking:
#         tracks["collab"] = trackers["collab"](
#             detections=collab_dets,
#             platform=platforms[ego_agent],
#         )
#         tracks_all["collab"].append(tracks["collab"])

#     ###############################################
#     # CENTRALIZED TRACKING USING DISTRIBUTED PERCEP
#     ###############################################
#     # run central tracker on all detections
#     if found_data:
#         if run_centralized_tracking:
#             tracks["central"] = trackers["central"](
#                 detections=dets,
#                 fovs=fovs,
#                 platforms=platforms,
#             )
#             tracks_all["central"].append(tracks["central"])

## Visualize Results